In [3]:
import glob
from music21 import converter, instrument, note, chord

In [10]:
!unzip "/content/drive/MyDrive/Colab Notebooks/Music Generation/midi_songs.zip" -d "/content/drive/MyDrive/Colab Notebooks/Music Generation/"

Archive:  /content/drive/MyDrive/Colab Notebooks/Music Generation/midi_songs.zip
  inflating: /content/drive/MyDrive/Colab Notebooks/Music Generation/midi_songs/0fithos.mid  
  inflating: /content/drive/MyDrive/Colab Notebooks/Music Generation/midi_songs/8.mid  
  inflating: /content/drive/MyDrive/Colab Notebooks/Music Generation/midi_songs/ahead_on_our_way_piano.mid  
  inflating: /content/drive/MyDrive/Colab Notebooks/Music Generation/midi_songs/AT.mid  
  inflating: /content/drive/MyDrive/Colab Notebooks/Music Generation/midi_songs/balamb.mid  
  inflating: /content/drive/MyDrive/Colab Notebooks/Music Generation/midi_songs/bcm.mid  
  inflating: /content/drive/MyDrive/Colab Notebooks/Music Generation/midi_songs/BlueStone_LastDungeon.mid  
  inflating: /content/drive/MyDrive/Colab Notebooks/Music Generation/midi_songs/braska.mid  
  inflating: /content/drive/MyDrive/Colab Notebooks/Music Generation/midi_songs/caitsith.mid  
  inflating: /content/drive/MyDrive/Colab Notebooks/Music Ge

In [13]:
notes=[]
for file in glob.glob("/content/drive/MyDrive/Colab Notebooks/Music Generation/midi_songs/*.mid"):
  midi=converter.parse(file)
  parts=instrument.partitionByInstrument(midi)
  notes_to_parse=parts.parts[0]
  for element in notes_to_parse:
    if(isinstance(element, note.Note)):
      # print(str(element.pitch))
      notes.append(str(element.pitch))
    elif(isinstance(element, chord.Chord)):
      # print(str(element.normalOrder))
      notes.append('.'.join(str(n) for n in element.normalOrder))

Streaming output truncated to the last 5000 lines.
C3
F3
F4
C4
[7, 10]
[9, 0]
[7, 10]
A5
[5, 9]
A5
E-2
G5
B-2
F#5
E-3
G5
G3
B-5
B-3
A5
G5
G5
D2
A2
F5
D3
F3
A3
D5
F5
G5
G2
D3
G3
B3
D4
A3
B3
D4
G4
C5
A5
G5
[0, 5]
[5, 9, 0]
[7, 9]
C4
[9, 0]
[5, 9]
C5
C5
A5
G5
[5, 9]
F2
[10, 1]
C#3
F3
C#4
B-3
[10, 1]
C4
[0, 3]
[10, 1]
F3
A4
C5
F2
C3
F3
[7, 9, 0]
F4
E5
F5
A5
C6
E6
C6
F2
C3
F3
F6
A3
C5
A5
G5
[5, 9, 0]
F2
[9, 0]
C3
F3
[7, 9, 0]
C5
A5
G5
[9, 0, 4]
A2
[9, 0, 4]
E3
A3
[11, 0, 4]
F5
[9, 0]
F6
E6
D6
[5, 10]
B-2
D3
F3
B-3
[5, 9]
A2
D3
F3
A3
[7, 10, 2]
G2
D3
G3
B-3
[2]
[2, 7]
[10]
[9]
[10, 2]
G5
G2
D4
D3
G3
[9, 10, 2]
[2]
G3
[10]
[2, 5]
[9]
[7, 10]
[2, 7]
[7, 0]
E4
C4
D4
B-3
C5
[7]
[5]
C3
[7]
[5, 9, 0]
F2
C3
F3
F4
C4
[7, 10]
[9, 0]
[7, 10]
[9, 0]
C6
[5, 9]
[9, 0]
E-2
[7, 10]
B-2
[6, 9]
E-3
[7, 10]
G3
[10, 2]
B-3
[9, 0]
[7, 10]
[7, 10]
D2
A2
[5, 9]
D3
F3
A3
[2, 5]
[5, 9]
[11]
G5
G2
D3
G3
B3
D4
A3
B3
D4
G4
G5
G6
D5
E5
[2, 5, 9]
[0, 4, 7]
A5
[9, 2]
C5
D5
E5
F5
[9, 0]
B5
[7, 11]
C6
[11, 4]
[9, 0, 4]
G5


In [14]:
len(notes)

57177

In [19]:
import numpy as np
from keras.utils import np_utils

In [16]:
n_vocab=len(set(notes))
sequence_length=100

In [17]:
pitch_names=sorted(set(item for item in notes))
note_to_int=dict((note, number) for number, note in enumerate(pitch_names))

In [18]:
network_input=[]
network_output=[]
for i in range(0, len(notes)-sequence_length):
  sequence_in=notes[i:i+sequence_length]
  sequence_out=notes[i+sequence_length]
  network_input.append([note_to_int[char] for char in sequence_in])
  network_output.append(note_to_int[sequence_out])

In [21]:
n_patterns=len(network_input)
network_input=np.reshape(network_input, (n_patterns,sequence_length,1))
network_input=network_input/float(n_vocab)
network_output=np_utils.to_categorical(network_output)
print(network_input.shape)

(57077, 100, 1)


In [25]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation

In [26]:
model=Sequential()
model.add(LSTM(256, input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 256)          264192    
_________________________________________________________________
dropout (Dropout)            (None, 100, 256)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 512)          1574912   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dense (Dense)                (None, 256)               65792     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0

In [28]:
# model.fit(network_input, network_output, epochs=80, batch_size=32)

Epoch 1/80
  71/1784 [>.............................] - ETA: 1:17:36 - loss: 5.2894

KeyboardInterrupt: ignored

In [ ]:
# model.save_weights('Music_125.h5')

In [29]:
from keras.models import load_model, model_from_json

In [30]:
with open('/content/drive/MyDrive/Colab Notebooks/Music Generation/music.json', 'r') as f:
  model=model_from_json(f.read())

In [31]:
model.load_weights('/content/drive/MyDrive/Colab Notebooks/Music Generation/music125.h5')

In [32]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
dropout_3 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100, 512)          1574912   
_________________________________________________________________
dropout_4 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)              

In [96]:
start=np.random.randint(0,len(network_input)-1)
start

16429

In [97]:
int_to_note=dict((number,note) for number, note in enumerate(pitch_names))

In [98]:
pattern=(network_input[start])
prediction_output=[]

In [99]:
for note_index in range(250):
  prediction_input=np.reshape(pattern, (1,len(pattern),1))
  prediction_input=(prediction_input)/float(n_vocab)
  prediction=model.predict(prediction_input, verbose=0)
  index=np.argmax(prediction)
  result=int_to_note[index]
  prediction_output.append(result)
  pattern=np.append(pattern, index)
  pattern=pattern[1:len(pattern)]

In [100]:
len(prediction_output)

250

In [112]:
offset=0
output_notes=[]
for pattern in prediction_output:
  if('.' in pattern) or pattern.isdigit():
    notes_in_chord=pattern.split('.')
    notes=[]
    for current_note in notes_in_chord:
      new_note=note.Note(int(current_note))
      new_note.storedInstrument=instrument.Piano()
      notes.append(new_note)
    new_chord=chord.Chord(notes)
    new_chord.offset=offset
    output_notes.append(new_chord)
  else:
    new_note=note.Note(pattern)
    new_note.offset=offset
    new_note.storedInstrument=instrument.Piano()
    output_notes.append(new_note)
  offset+=0.5

In [113]:
import music21

In [114]:
midi_stream=music21.stream.Stream(output_notes)
midi_stream.write('midi', fp='Piano song.mid')

'Piano song.mid'